In [1]:
import torch
from torch import nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from pickle import dump

In [2]:
data = pd.read_csv('train_with_out_vanishing.csv')

In [3]:
data.head()

,Positions X
0,-5216.84
1,-5216.01
2,-5213.73
3,-5210.01
4,-5204.82


In [4]:
data.shape

(624, 1)

In [5]:
data.drop_duplicates(inplace= True)

In [6]:
data.shape

(624, 1)

In [7]:
scaler = MinMaxScaler(feature_range = (0,1)) # escalando os dados
data_scaled = scaler.fit_transform(data)
dump(scaler, open('scaler.pkl', 'wb')) # salvando o scaler.

In [8]:
last_positions = [] # lista que irá armazenar as últimas 10 posições.
real_position = [] # lista que irá armazenar as posições reais.
for i in range(10, 624):
    last_positions.append(data_scaled[i-10:i, 0])
    real_position.append(data_scaled[i, 0])

In [9]:
last_positions, real_position = np.array(last_positions), np.array(real_position)

In [10]:
last_positions = np.reshape(last_positions, (last_positions.shape[0], last_positions.shape[1], 1))

In [11]:
last_positions = torch.from_numpy(last_positions).type(torch.Tensor)
real_position = torch.from_numpy(real_position).type(torch.Tensor)

In [12]:
input_dim = 1
hidden_dim = 100
num_layers = 1
output_dim = 1
num_epochs = 50

In [15]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.gru(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [16]:
model = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.RMSprop(model.parameters(), lr=0.01)

In [17]:
import time
hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []
for t in range(num_epochs):
    y_train_pred = model(last_positions)    
    loss = criterion(y_train_pred, real_position)
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()    
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    
training_time = time.time() - start_time
print("Training time: {}".format(training_time))

AttributeError: 'tuple' object has no attribute 'size'

In [106]:
data_test = pd.read_csv('test_with_out_vanishing .csv')

In [107]:
data_test.head()

,Positions X
0,4680.74
1,4696.74
2,4712.74
3,4728.74
4,4744.74


In [108]:
data_test.shape

(31, 1)

In [109]:
data_completed = pd.concat((data, data_test), axis = 0)

In [110]:
data_completed.shape

(655, 1)

In [111]:
inputs = data_completed[len(data_completed) - len(data_test) - 10:].values

In [112]:
inputs = inputs.reshape(-1, 1)

In [113]:
inputs = scaler.transform(inputs)

In [114]:
X_test = []
for i in range(10, 41):
    X_test.append(inputs[i - 10:i, 0])

In [115]:
X_test = np.array(X_test)

In [116]:
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_test = torch.from_numpy(X_test).type(torch.Tensor)

In [117]:
model.eval()
with torch.no_grad():
    predict = model(X_test)

In [118]:
predict = scaler.inverse_transform(predict)

In [119]:
predict[0]

array([-320.46853498])

In [120]:
data_test.head()

,Positions X
0,4680.74
1,4696.74
2,4712.74
3,4728.74
4,4744.74
